In [ ]:
import sys
sys.path.append('../evapo_scripts/Auxiliary/')
from helpsters import *

In [ ]:
# workaround with relative path mess from containers
os.chdir('/data/Aldhani/eoagritwin/')
# get rid of force output that is not needed -> months outside of growing season that do not exist in AI4Boundaries
files = getFilelist('force/output/BRANDENBURG/', '.tif', deep=True)

filesToKill = [f for f in files if int(f.split('-')[-1].split('.')[0]) not in [i for i in range(3,9,1)]]
for file in filesToKill:
    RasterKiller(file)

files = list(filter(lambda item: item not in filesToKill, files))

In [29]:
# bring files in list in order
tiles = list(set([file.split('output/')[-1].split('/')[1].split('/')[0] for file in files]))
tilefilesL = []
for tile in tiles:
    tilefiles = [file for file in files if tile in file]
    tilefilesL.append(getBluGrnRedBnrFORCEList(tilefiles))

# create stacked vrts of tiles
# make folder for tiles
outDir = f'fields/Auxiliary/vrt/'#{getFORCExyRange(tiles)}/'
os.makedirs(outDir, exist_ok=True)

for i in range(len(tilefilesL[0])):
    vrt = gdal.BuildVRT(f'{outDir}{getFORCExyRange(tiles)}_{str(i)}.vrt', [tilefilesL[0][i], tilefilesL[1][i]], separate = False)
    vrt = None

vrts = getFilelist(outDir, '.vrt')
nums = [int(vrt.split('_')[-1].split('.')[0]) for vrt in vrts]
vrts_sorted = sortListwithOtherlist(nums, vrts)

vrt = gdal.BuildVRT(f'{outDir}{getFORCExyRange(tiles)}_Cube.vrt', vrts_sorted, separate = True)
vrt = None

# clean up 1d vrts
for vrt in vrts:
    RasterKiller(vrt)

In [30]:
# make vrt of predicted image chips
chips = getFilelist('/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/chips/', '.tif')

# create stacked vrts of chips
vrt = gdal.BuildVRT('/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/vrt/chipsvrt.vrt', chips, separate = False)
vrt = None


In [31]:
# make vrt of masked predicted image chips
chips = getFilelist('/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/masked_chips/', '.tif')

# create stacked vrts of masked chips
vrt = gdal.BuildVRT('/data/Aldhani/eoagritwin/fields/output/predictions/FORCE/vrt/masked_chipsvrt.vrt', chips, separate = False)
vrt = None
